<a href="https://colab.research.google.com/github/ErzAP-maker/erza_ap-public/blob/main/import_xlsx_from_ZIP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import zipfile
import pandas as pd
from datetime import datetime, timedelta
import shutil

# Tentukan folder input dan output
input_folder = '/content/drive/MyDrive/Incoming Chat '  # Folder tempat file zip disimpan
output_folder = '/content/drive/MyDrive/Incoming Chat /Incoming Chat 2024 [new]'  # Folder untuk menyimpan file xlsx
archive_folder = '/content/drive/MyDrive/Incoming Chat '  # Folder untuk memindahkan file zip yang sudah diproses

# Pastikan folder output dan archive ada
os.makedirs(output_folder, exist_ok=True)
os.makedirs(archive_folder, exist_ok=True)

# Mendapatkan waktu 24 jam yang lalu
time_24_hours_ago = datetime.now() - timedelta(days=1)

# Fungsi untuk memeriksa file ZIP yang baru ditambahkan dalam 24 jam terakhir
def get_recent_zip_files(input_folder):
    recent_files = []
    for file_name in os.listdir(input_folder):
        file_path = os.path.join(input_folder, file_name)
        if file_name.endswith('.zip') and os.path.getmtime(file_path) > time_24_hours_ago.timestamp():
            recent_files.append(file_path)
    return recent_files

# Fungsi untuk mengekstrak CSV dari ZIP dan mengonversinya ke XLSX
def process_zip(zip_file_path):
    # Buka file zip
    with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
        # Daftar file dalam zip
        file_list = zip_ref.namelist()

        # Cari file CSV yang mengandung "Raw_Incoming_Chat_2024"
        for file_name in file_list:
            if 'Raw_Incoming_Chat_2024' in file_name and file_name.endswith('.csv'):
                # Ekstrak file CSV
                csv_file_path = os.path.join('/content', file_name)
                zip_ref.extract(file_name, '/content')

                # Baca CSV menjadi DataFrame
                df = pd.read_csv(csv_file_path)

                # Tentukan nama file XLSX
                xlsx_file_name = file_name.replace('.csv', '.xlsx')
                xlsx_file_path = os.path.join(output_folder, xlsx_file_name)

                # Simpan ke XLSX
                df.to_excel(xlsx_file_path, index=False)

                # Hapus file CSV setelah konversi
                os.remove(csv_file_path)
                print(f"File {file_name} berhasil diproses dan disimpan sebagai {xlsx_file_name}")
                return True
    return False

# Ambil file zip yang baru ditambahkan dalam 24 jam terakhir
recent_zip_files = get_recent_zip_files(input_folder)

# Proses setiap file zip yang baru
for zip_file in recent_zip_files:
    if process_zip(zip_file):
        # Pindahkan file zip yang sudah diproses ke folder archive
        shutil.move(zip_file, os.path.join(archive_folder, os.path.basename(zip_file)))
        print(f"File zip {os.path.basename(zip_file)} dipindahkan ke folder archive.")


File Raw_Incoming_Chat_2024-11-12.csv berhasil diproses dan disimpan sebagai Raw_Incoming_Chat_2024-11-12.xlsx
File zip IncomingChatData_2024-11-12.zip dipindahkan ke folder archive.
